In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
# Fix export PDF as text issue
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

fpath = join(plt.rcParams["datapath"], "fonts/ttf/arial.ttf")
from matplotlib import font_manager as fm, rcParams
prop = fm.FontProperties(fname=fpath)
rcParams["font.family"] = "arial"

import os
import scanpy as sc
from os.path import join, exists
from os import listdir
import anndata
import scipy
import numpy as np
import sys


import seaborn as sns
sns.set()
from matplotlib import rcParams
import matplotlib.pyplot as plt

from utils import *

# convert counts into float32
# Convenience method for computing the size of objects
def print_size_in_MB(x):
    return '{:.3} MB'.format(x.__sizeof__()/1e6)

def print_size_in_MB_sparse_matrix(a):
    # a = scipy.sparse.csr_matrix(np.random.randint(10, size=(40, 3)))
    # x = a.data.nbytes + a.indptr.nbytes + a.indices.nbytes
    size = a.data.size/(1024**2)
    return '{:.3} MB'.format(size)

import warnings
warnings.filterwarnings("ignore")

In [47]:
ls -ltrh ../../data/integration_march_2021/input/

total 14G
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 29 08:48 bydataset_500/
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 29 09:19 bydataset_1000/
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 435M Apr 29 09:41 input_500_cells.h5ad
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User 863M Apr 29 09:43 input_1000_cells.h5ad
drwxr-xr-x. 2 ignacio.ibarra OG-ICB-User   11 Apr 29 09:54 bydataset/
-rw-r--r--. 1 ignacio.ibarra OG-ICB-User  12G Apr 29 11:47 input_all_cells.h5ad


In [49]:
input_dir = '../../data/integration_march_2021/input'

In [212]:
ad_by_f['input_500_cells.h5ad']

AnnData object with n_obs × n_vars = 69627 × 3840
    obs: 'batch', 'dataset', 'filename', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'size_factors', 'cell.type', 'batch.merged'
    layers: 'counts'

In [208]:
ad_by_f.keys()

dict_keys(['input_500_cells.h5ad', 'input_1000_cells.h5ad', 'input_all_cells.h5ad'])

In [229]:
ad_by_f = {}
for f in listdir(input_dir):
    if not f.endswith('h5ad'):
        continue
    print(f)
    p = join(input_dir, f)
    ad = sc.read_h5ad(p)
    ad_by_f[f] = ad
    print('done...')

input_500_cells.h5ad
done...
input_1000_cells.h5ad
done...
input_all_cells.h5ad
done...


In [231]:
for f in ad_by_f:
    print(f)
    
    ad = ad_by_f[f]
    import pandas as pd

    ad.obs['batch.full'] = ad.obs['dataset'].astype(str) + '_' + ad.obs['batch.merged'].astype(str)
    df = ad.obs[['cell.type', 'batch.full']].value_counts().reset_index()
    df = df.pivot('batch.full', 'cell.type', 0)

    name = df.index
    df = df.reset_index(drop=True)
    df = pd.DataFrame(np.array(df.fillna(0)).astype(int), columns=list(df.columns), index=list(name))

    df['sum'] = df.sum(axis=1)
    df['k'] = ['_'.join(a) for a in df.index.str.split('_').str[:-1]]
    
    df = df.sort_values(['k','sum'], ascending=[False, False])
    del df['sum']
    del df['k']
    
    rcParams['figure.figsize'] = [15, 7]
    rcParams['figure.dpi'] = 120

    from itertools import cycle, islice
    import pandas, numpy as np  # I find np.random.randint to be better
    c = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', 'lightgrey']
    my_colors = list(islice(cycle(c), None, len(df)))

    for pi in [0, 1]:
        ax = plt.subplot(2, 1, pi + 1)
        df.plot(kind='bar', stacked=True, width=1.0, linewidth=0.2, logy=pi == 0,
                color=my_colors, edgecolor='black', ax=ax)
        plt.ylabel('# cells%s' % (' (log)' if pi == 0 else ''))
        plt.xlabel('dataset batch')
        plt.xticks(fontsize=6)
        plt.legend(fontsize=10, title='scpred', bbox_to_anchor=(1.01, 1))
        
    
    img_path = '../../scIB_run/%s' % f.replace('.h5ad', '_barplot_scpred.png')
    plt.savefig(img_path)
    
    print(os.path.abspath(img_path)) # .replace('.pdf', '.png')))
    plt.close()


input_500_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/scIB_run/input_500_cells_barplot_scpred.png
input_1000_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/scIB_run/input_1000_cells_barplot_scpred.png
input_all_cells.h5ad
/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/scIB_run/input_all_cells_barplot_scpred.png


In [233]:
print(os.path.abspath(img_path))

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/scIB_run/input_all_cells_barplot_scpred.png


In [235]:
%matplotlib inline

In [236]:
from PIL import Image

image = Image.open(img_path)
image.show()